# Modul 6 - Projektaufgabe
## Ugly is better than nothing - Generieren von Titelaufnahmen aus den METS/MODS-Daten digitalisierter Handschriften zur Übernahme in den K10plus und Nachnutzung im lokalen Nachweissystem ACTApro
### Ulrike Mehringer, 4.7.2023-18.8.2023

In [1]:
import urllib.request
import xml.etree.ElementTree as ET
import re

## Personen: von GND zu PPN

In [2]:
# Personennormsatz-ID (PPN) über die GND-Nummer im K10Plus abrufen
# gnd_nr = "11879650X"
gnd_nr = "118655477"
def gnd_ppn(gnd_nr):
    url = "http://sru.k10plus.de/opac-de-627!rec=2?&operation=searchRetrieve&query=pica.nid="+gnd_nr+"&maximumRecords=1&recordSchema=mods"
    person_xml = "../data/person.xml"
    urllib.request.urlretrieve(url,person_xml)
# XML parsen
    person_tree = ET.parse(person_xml)
    person_root = person_tree.getroot()
# K10Plus-PPN aus person_file auslesen
    rec_id = person_root.find('.//{http://www.loc.gov/mods/v3}recordIdentifier')
# Variable für die Textausgabe
    ppn = rec_id.text
    return(ppn)
    

In [3]:
# print(person_root.findall(".//{http://docs.oasis-open.org/ns/search-ws/sruResponse}recordData/{http://www.loc.gov/mods/v3}mods/*"))

In [4]:
# rec_id = person_root.find('.//{http://www.loc.gov/mods/v3}recordIdentifier')

In [5]:
# type(rec_id)

In [6]:
# print(rec_id.attrib)

In [7]:
# print(rec_id.text)

In [8]:
# Variable für die Textausgabe
# ppn = rec_id.text
# return(ppn)

In [9]:
# print(ppn)

## METS/MODS aus OpenDigi

In [10]:
# METS-Daten des OpenDigi-Projekts einlesen
od_projekt = "Mc339"
url = "https://idb.ub.uni-tuebingen.de/opendigi/"+od_projekt+"/mets"
od_xml = "../data/"+od_projekt+".xml"
urllib.request.urlretrieve(url,od_xml)

('../data/Mc339.xml', <http.client.HTTPMessage at 0x7ff6b8046550>)

In [11]:
od_tree = ET.parse(od_xml)
od_root = od_tree.getroot()

In [12]:
od_data = od_root.find(".//{http://www.loc.gov/METS/}xmlData/*")

In [13]:
type(od_data)

xml.etree.ElementTree.Element

In [14]:
for child in od_data:
    print(child.tag,child.attrib)

{http://www.loc.gov/mods/v3}titleInfo {}
{http://www.loc.gov/mods/v3}identifier {'type': 'urn'}
{http://www.loc.gov/mods/v3}recordInfo {}
{http://www.loc.gov/mods/v3}location {}
{http://www.loc.gov/mods/v3}language {}
{http://www.loc.gov/mods/v3}name {'authority': 'gnd', 'authorityURI': 'http://d-nb.info/gnd/', 'type': 'personal', 'valueURI': 'http://d-nb.info/gnd/119244071'}
{http://www.loc.gov/mods/v3}name {'type': 'personal'}
{http://www.loc.gov/mods/v3}name {'type': 'personal'}
{http://www.loc.gov/mods/v3}originInfo {}


In [15]:
# Titel, Feld 4000
od_title = od_root.find(".//{http://www.loc.gov/mods/v3}title")
print(od_title.text)

Novella super quarto libro decretalium Johannis Andreae


In [16]:
# URN, Feld 2050 und 4950
od_urn = od_root.find(".//{http://www.loc.gov/mods/v3}recordIdentifier")
print(od_urn.text)

urn:nbn:de:bsz:21-dt-258226


In [17]:
# Signatur, Feld 4065 $a
od_shelf = od_root.find(".//{http://www.loc.gov/mods/v3}shelfLocator")
print(od_shelf.text)

Mc 339


In [18]:
# Sprache, Feld 1500
od_lang = od_root.find(".//{http://www.loc.gov/mods/v3}languageTerm")
if re.search(r"; *",od_lang.text):
    od_lang = str_replace("; ","$a",od_lang.text)
print(od_lang.text)

lat


In [19]:
# Entstehungsort, Feld 4046 (Feld 4040?)
od_place = od_root.find(".//{http://www.loc.gov/mods/v3}placeTerm")
print(od_place.text)
if re.search(r"[oO]. *[oO.]",od_place.text):
    od_place = "[Entstehungsort nicht ermittelbar]"
print(od_place) 

o.O.
[Entstehungsort nicht ermittelbar]


In [20]:
# Entstehungsdatum, Feld 1100, 1100 $n
od_date = od_root.findall(".//{http://www.loc.gov/mods/v3}dateIssued")
# od_date ist eine Liste
od_date_norm = od_date[0].text
od_date_subm = "["+od_date[1].text+"]"
if re.search(r"[oO]. *[dD].",od_date_subm):
    od_date_norm = "[1XXX]"
    od_date_subm = "[Entstehungdatum nicht ermittelbar]"
print(od_date_norm,od_date_subm)

1393 [um 1393]


In [21]:
# Personen, Feld 3000, 3010
od_name = od_root.findall(".//{http://www.loc.gov/mods/v3}name")
od_name

[<Element '{http://www.loc.gov/mods/v3}name' at 0x7ff6b8047ea0>,
 <Element '{http://www.loc.gov/mods/v3}name' at 0x7ff6a27c2130>,
 <Element '{http://www.loc.gov/mods/v3}name' at 0x7ff6a27c2270>]

In [22]:
gnd_url = od_name[0].attrib['valueURI']
gnd_nr = str(re.findall(r'/gnd/(.*)',gnd_url))
print(gnd_url,gnd_nr)
gnd_ppn(gnd_nr)

http://d-nb.info/gnd/119244071 ['119244071']


'080046096'

In [62]:
for name in od_name:
    y = str(name.attrib)
    if 'valueURI' in y:
        gnd_url = name.attrib['valueURI']
        gnd_nr = str(re.findall(r'/gnd/(.*)',gnd_url))
        name_ppn = gnd_ppn(gnd_nr)
        print(name_ppn)
    else:
        print("huhu")
        od_name_display = od_root.findall(".//{http://www.loc.gov/mods/v3}displayForm")
        print(od_name_display[0].text)
        print(od_name_display[1].text)
        print(od_name_display[2].text)

080046096
huhu
Johannes Andreae
Hieronymus Angeli de Esslingen [Schreiber]
Meier de Wasszneck, Ulrich [Schreiber]
huhu
Johannes Andreae
Hieronymus Angeli de Esslingen [Schreiber]
Meier de Wasszneck, Ulrich [Schreiber]


In [28]:
od_name_display = od_root.findall(".//{http://www.loc.gov/mods/v3}displayForm")
od_name_display

[<Element '{http://www.loc.gov/mods/v3}displayForm' at 0x7ff6b8047f40>,
 <Element '{http://www.loc.gov/mods/v3}displayForm' at 0x7ff6a27c2180>,
 <Element '{http://www.loc.gov/mods/v3}displayForm' at 0x7ff6a27c22c0>]

In [33]:
od_name_display[0].text

'Johannes Andreae'

In [34]:
od_name_display[1].text

'Hieronymus Angeli de Esslingen [Schreiber]'

In [35]:
od_name_display[2].text

'Meier de Wasszneck, Ulrich [Schreiber]'